In [1]:
import import_ipynb
from RC_Transformer import transformer
from RC_Transformer import CustomSchedule
from CommonModule.Handle_Dir import mkdir_p, del_folder
from CommonModule.Encoder import IntegerEncoder
from CommonModule.Decoder import Decoder

import re
import os
import numpy as np
from pathlib import Path
from glob import iglob
import tensorflow as tf
import sentencepiece as spm
import csv
import pandas as pd

importing Jupyter notebook from RC_Transformer_split_vocab.ipynb
importing Jupyter notebook from /data/ksb/TestSampleDir/articleSummary-Jupyter/Transformer/CommonModule/Handle_Dir.ipynb
importing Jupyter notebook from /data/ksb/TestSampleDir/articleSummary-Jupyter/Transformer/CommonModule/Encoder.ipynb
importing Jupyter notebook from /data/ksb/TestSampleDir/articleSummary-Jupyter/Transformer/CommonModule/Decoder.ipynb


In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    except RuntimeError as e:
        print(e)

In [3]:
BASE_DIR = "/data/ksb"
DATA_BASE_DIR = os.path.join(BASE_DIR, 'sample_articles')
SRC_BASE_DIR = os.path.join(BASE_DIR, 'TestSampleDir')

VAL_PREPROCESSED_PATH = os.path.join(DATA_BASE_DIR,"Valid-Preprocessed-Data")
VAL_SUMMARY_PREPROCESSED_PATH = os.path.join(DATA_BASE_DIR,"Valid-Summary-Preprocessed-Data")

PREPROCESSED_PATH = os.path.join(DATA_BASE_DIR,"Preprocessed-Data")
SUMMARY_PREPROCESSED_PATH = os.path.join(DATA_BASE_DIR,"Summary-Preprocessed-Data")

PREDICT_PATH = os.path.join(DATA_BASE_DIR,"Predict-Data-Split-Vocab")

WORD_ENCODING_DIR = os.path.join(os.path.join(SRC_BASE_DIR, 'articleSummary-Jupyter'), 'Word-Encoding-Model')
MODEL_DIR = os.path.join(os.path.join(SRC_BASE_DIR, 'articleSummary-Jupyter'), 'Transformer-Model')


In [4]:
D_MODEL = 128

In [5]:
sp = spm.SentencePieceProcessor()
model_num = len(list(iglob(os.path.join(WORD_ENCODING_DIR, 'spm-input-*.vocab'), recursive=False))) -1
sp.Load(os.path.join(WORD_ENCODING_DIR, 'spm-input-{}.model').format(model_num))


True

In [6]:
VOCAB_SIZE = 70000

In [7]:
options = {
    'model-type' : 'Sentence-Piece',
    'inv_wv' : None,
    'corpus' : None,
    'spm' : sp
}
input_encoded_list = IntegerEncoder(options=options, filepaths=list(iglob(os.path.join(PREPROCESSED_PATH, '**.csv'), recursive=False))).encoder()
output_encoded_list = IntegerEncoder(options=options, filepaths=list(iglob(os.path.join(SUMMARY_PREPROCESSED_PATH, '**.csv'), recursive=False))).encoder()


In [8]:
LAYER_NUM = 6
NUM_HEADS = 8
DFF = 512

BATCH_SIZE = 64
BUFFER_SIZE = 5000

WARMUP_STEPS = 50
EPOCHS = 30

In [9]:
START_TOKEN = [sp.bos_id()]
END_TOKEN = [sp.eos_id()]

In [10]:
get_max_length = lambda x : np.max([len(line) for line in x])

MAX_LEN = get_max_length(input_encoded_list)
MAX_LEN

250

In [11]:
SUMMARY_MAX_LEN = get_max_length(output_encoded_list)
SUMMARY_MAX_LEN

190

In [12]:
input_encoded_list = list(map(lambda list_ : START_TOKEN + list_ + END_TOKEN, input_encoded_list))
output_encoded_list = list(map(lambda list_ : START_TOKEN + list_ + END_TOKEN, output_encoded_list))

In [13]:
from sklearn.model_selection import train_test_split
input_train, input_test, output_train, output_test = train_test_split(
    input_encoded_list, output_encoded_list, test_size=0.2, random_state=42)

In [14]:
MAX_LEN += 2

In [15]:
SUMMARY_MAX_LEN +=2

In [16]:
# Padding
train_input_encoded_matrix = tf.keras.preprocessing.sequence.pad_sequences(
    input_train, maxlen=MAX_LEN, padding='post')
train_summary_encoded_matrix = tf.keras.preprocessing.sequence.pad_sequences(
    output_train, maxlen=MAX_LEN, padding='post')

In [17]:
# Padding
test_input_encoded_matrix = tf.keras.preprocessing.sequence.pad_sequences(
    input_test, maxlen=MAX_LEN, padding='post')
test_summary_encoded_matrix = tf.keras.preprocessing.sequence.pad_sequences(
    output_test, maxlen=MAX_LEN, padding='post')

In [18]:
print('Train Contents Shape : {}'.format(train_input_encoded_matrix.shape))
print('Train Summaries Shape : {}'.format(train_summary_encoded_matrix.shape))
print('Test Contents Shape : {}'.format(test_input_encoded_matrix.shape))
print('Test Summaries Shape : {}'.format(test_summary_encoded_matrix.shape))

Train Contents Shape : (22658, 252)
Train Summaries Shape : (22658, 252)
Test Contents Shape : (5665, 252)
Test Summaries Shape : (5665, 252)


In [19]:
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': train_input_encoded_matrix, # Encoder Input
        'dec_inputs': train_summary_encoded_matrix[:, :-1] # Decoder Input
    },
    {
        # Decoder Output, Remove <SOS>
        'outputs': train_summary_encoded_matrix[:, 1:]  
    },
))

In [20]:
dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [21]:
val_dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': test_input_encoded_matrix, # Encoder Input
        'dec_inputs': test_summary_encoded_matrix[:, :-1] # Decoder Input
    },
    {
        # Decoder Output, Remove <SOS>
        'outputs': test_summary_encoded_matrix[:, 1:]  
    },
))
val_dataset = val_dataset.cache()
val_dataset = val_dataset.shuffle(BUFFER_SIZE)
val_dataset = val_dataset.batch(BATCH_SIZE)
val_dataset = val_dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [22]:
lrate_scheduler = CustomSchedule(d_model=D_MODEL)

In [23]:
beta_1 = 0.9  
beta_2 = 0.98
epsilon = 10 ** -9

optimizer = tf.keras.optimizers.Adam(lrate_scheduler, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

In [24]:
with tf.device('/CPU'):
    model = transformer(
        vocab_size=VOCAB_SIZE,
        num_layers=LAYER_NUM,
        dff=DFF,
        d_model=D_MODEL,
        num_heads=NUM_HEADS,
        dropout = 0.3)

(None, None, 128)
(None, None, 128)


In [25]:
def loss_function(y_true, y_pred):
    y_true = tf.reshape(y_true, shape=(-1, MAX_LEN-1))

    loss = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none')(y_true, y_pred)

    mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
    loss = tf.multiply(loss, mask)

    return tf.reduce_mean(loss)

In [26]:
checkpoint_dirpath = os.path.join(MODEL_DIR, "RC_Transformer")

mkdir_p(checkpoint_dirpath)
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_dirpath,
    save_weights_only=True,
    monitor='loss',
    mode='max',
    save_best_only=True)

In [27]:
from tensorflow.keras.losses import sparse_categorical_crossentropy

model.compile(optimizer=optimizer, loss=loss_function)
model.summary()

Model: "transformer"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 128)    8960000     inputs[0][0]                     
__________________________________________________________________________________________________
tf.math.multiply_1 (TFOpLambda) (None, None, 128)    0           embedding_1[0][0]                
__________________________________________________________________________________________________
positional_encoding_1 (Position (None, None, 128)    0           tf.math.multiply_1[0][0]         
________________________________________________________________________________________

In [28]:
model.fit(dataset, batch_size=BATCH_SIZE, epochs=15, verbose=2, validation_data=val_dataset, 
          shuffle=True, callbacks=[model_checkpoint_callback])


Epoch 1/15
355/355 - 317s - loss: 1.5258 - val_loss: 1.3763
Epoch 2/15
355/355 - 288s - loss: 1.3307 - val_loss: 1.3132
Epoch 3/15
355/355 - 290s - loss: 1.2879 - val_loss: 1.2682
Epoch 4/15
355/355 - 293s - loss: 1.2219 - val_loss: 1.2021
Epoch 5/15
355/355 - 295s - loss: 1.1563 - val_loss: 1.1461
Epoch 6/15
355/355 - 295s - loss: 1.0932 - val_loss: 1.0895
Epoch 7/15
355/355 - 294s - loss: 1.0276 - val_loss: 1.0364
Epoch 8/15
355/355 - 291s - loss: 0.9646 - val_loss: 0.9860
Epoch 9/15
355/355 - 286s - loss: 0.9019 - val_loss: 0.9453
Epoch 10/15
355/355 - 287s - loss: 0.8315 - val_loss: 0.8775
Epoch 11/15
355/355 - 282s - loss: 0.7279 - val_loss: 0.7225
Epoch 12/15
355/355 - 295s - loss: 0.5626 - val_loss: 0.5115
Epoch 13/15
355/355 - 295s - loss: 0.3884 - val_loss: 0.4579
Epoch 14/15
355/355 - 294s - loss: 0.2753 - val_loss: 0.3162
Epoch 15/15
355/355 - 294s - loss: 0.2034 - val_loss: 0.2794


In [33]:
option={
        'model-type' : 'Sentence-Piece',
        'inv_wv' : None,
        'corpus' : None,
        'spm' : sp
    }

In [34]:
def evaluate(content, max_len):
    
#     enc_input_sent = IntegerEncoder(options=src_option, filepaths=None).line_encoder(content)
#     enc_input_sent = START_TOKEN + enc_input_sent + END_TOKEN
    enc_input_sent= content
    enc_input_sent = tf.expand_dims(enc_input_sent, axis=0)
    
    output = START_TOKEN
    output = tf.expand_dims(output, axis=0)
            
    for i in range(max_len):
        predictions = model(inputs=[enc_input_sent, output], training=False)
        
        predictions = predictions[:, -1:, :]
        
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)
        predicted_id = tf.squeeze(predicted_id)
#         print("index : {idx}, predicted : {pre}".format(idx=i, pre=predicted_id))

        if tf.equal(predicted_id, END_TOKEN[0]):
            break

        output = tf.concat([output, tf.expand_dims([predicted_id], axis=0)], axis=-1)

    return tf.squeeze(output, axis=0)


In [35]:
def predict(content):
    prediction = evaluate(content, SUMMARY_MAX_LEN)

    predict_list = prediction.numpy().tolist()
    decoder = Decoder(option)
    predicted_sentence = decoder.decode(predict_list)

    return predicted_sentence

In [36]:
def saveCSVFile(baseDir, media, article_dist):
    save_path = os.path.join(baseDir, media) + ".csv"

    article_dist.to_csv(save_path, mode='w', header=False)

In [37]:
def get_media_name(filepath):
    filename = filepath.split(os.sep)[-1]
    return filename.split(".")[0]

In [38]:
del_folder(PREDICT_PATH)
mkdir_p(PREDICT_PATH)

src data (기사 본문과 비교)

#### ROUGE-L Score  
reference : 기사 본문  
produced summary : RC-Transformer 생성 요약문 

In [ ]:
import nltk.translate.bleu_score as bleu
from rouge import Rouge 
rouge = Rouge()

generated_summary_dist = pd.DataFrame(columns=['Origin Contents', 'Generated Summary', 
                                               'BLEU', 'ROUGE F1', 'ROUGE Recall', 'ROUGE Precision'])

for inp, outp in zip(input_test, output_test):
    decoder_src = Decoder(option)
    input_sent = decoder_src.decode(inp)
    
    predict_summary = predict(inp)
    
    print('Input: {}'.format(input_sent))
    print('Output: {}'.format(predict_summary))
    
#     ref_pcs = [sp_src.encode_as_pieces(input_sent)]
#     pred_pcs = [sp_src.encode_as_pieces(predict_summary)]
#     bleu_score = bleu.corpus_bleu([ref_pcs], pred_pcs)

    bleu_score = bleu.sentence_bleu([input_sent], predict_summary)
    print('BLEU: {}'.format(bleu_score * 100))
    
    # ROUGE-L Score
    rouge_scores = rouge.get_scores(predict_summary, input_sent)
    rouge_f = rouge_scores[0]['rouge-l']['f']
    print('Rouge F1: {}'.format(rouge_f * 100))
    
    rouge_r = rouge_scores[0]['rouge-l']['r']
    print('Rouge Recall: {}'.format(rouge_r * 100))
    
    rouge_p = rouge_scores[0]['rouge-l']['p']
    print('Rouge Precision: {}\n'.format(rouge_p * 100))
    
    summary = {'Origin Contents' : input_sent, 'Generated Summary' : predict_summary,
               'BLEU':bleu_score, 'ROUGE F1': rouge_f, 'ROUGE Recall': rouge_r, 'ROUGE Precision': rouge_p}
    
    generated_summary_dist = generated_summary_dist.append(summary, ignore_index=True)
    

Input: 경북 고령군 한 요양원에서 요양보호사와 원장이 노인 환자를 폭행한 것으로 드러나 경찰이 수사에 나섰습니다 경북 고령경찰서는 요양원 관계자들이 80대 환자를 폭행하는 장면이 담긴 CCTV 영상이 알려짐에 수사에 착수했다고 밝혔습니다 해당 CCTV 영상은 지난해 11월 찍힌 것으로 요양보호사가 노인 환자를 이불로 덮어씌운 뒤 마구 때리고 발로 차는 폭행 행위가 고스란히 담겨져 있습니다 요양원 원장이 같은 환자를 때리는 장면도 찍혔습니다 경찰은 요양원 관계자들을 불러 CCTV에 찍힌 내용을 토대로 폭행 여부를 확인할 계획이라고 말했습니다
Output: 해당 CCTV 영상은 지난해 11월 찍힌 것으로 확인되면서 노인 환자를 감싸는 침입하려 람정운 뒤 마구 때리고 발로 차는 폭행 행위가 고스란히 담겨 있다고 말했습니다
BLEU: 8.231378957961736
Rouge F1: 43.90243861035099
Rouge Recall: 31.03448275862069
Rouge Precision: 75.0

Input: 최근 한 특수학교에서 사회복무요원이 장애 학생을 폭행해 논란이 된 일이 있었는데요 특수학교에서 이번에는 교사들이 학생을 집단으로 폭행한 사실이 드러나 경찰이 수사에 나섰습니다 보도합니다 서울 강서구의 한 사립 특수학교 지적 장애 등을 가진 학생 백여 명이 재학중입니다 지난 7월말 학교에서 한 담임 교사가 엘리베이터에서 학생을 거칠게 잡아끄는 폭행했다는 고소장이 경찰에 접수됐습니다 경찰은 학부모 진술을 받은 뒤 먼저 학교 측 CCTV를 확인했습니다 화면에 담긴 모습들은 충격적이었습니다 담임 교사뿐 아니라 옆반 교사와 교육을 지원해주는 실무사까지 합세해 학생을 폭행하는 장면이 확인된 겁습니다 발로 차는 모습 문 사이에 끼워놓고 폭행하는 모습이 고스란히 찍혔습니다 경찰 조사 결과 현재까지 확인된 폭행이나 학대는 수십 건 피해 학생은 두 명 폭행에 가담한 교사는 십여 명이었습니다 경찰은 집단 폭행한 교사들에 대해 특수폭행 혐의 적용을 검토하는 한편 

Input: 경남 진주의 아파트 방화 사건 피의자는 주민인 40대 남성인데요 자신의 집에 불을 낸 뒤 대피하던 이웃들을 흉기로 공격했는데 최근까지 이웃과 갈등이 잦았던 것으로 확인됐습니다 현장 연결해 수사상황 알아보겠습니다 현재 수사가 진행되고 있습니까 저는 지금 수사본부가 마련된 경남 진주경찰서에 나와 있습니다 현재 경찰서에서는 아파트 방화 살인사건 피의자 42살 안 모씨에 대한 조사가 진행 중인데요 안 씨는 범행을 일부 시인했지만 자세한 사건 경위와 범행 동기를 두고 횡설수설하며 제대로 진술하지 않는 것으로 알려졌습니다 경남지방경찰청은 진주경찰서장을 총괄로 한 특별 수사단을 꾸려 형사 전원을 투입해 수사 중인데요 경찰 조사 결과 피의자 안 씨는 살았고 지난 2015년 조현병 증세로 치료받은 것으로 확인됐습니다 경찰은 프로파일러를 투입해 안 씨의 심리 상태를 파악하는 한편 안 씨에 대해 방화와 살인 살인미수 혐의로 구속영장을 신청할 예정입니다 피의자가 이웃과 마찰이 잦아 경찰도 여러 번 출동했다고 하던데 제대로 조치되지 않은 건가요 경찰은 안 씨와 관련해 최근 7건의 경찰 출동이 있었고 가운데 5건이 해당 아파트에서 벌어졌다고 밝혔는데요 층간소음이나 벌레가 떨어진다며 이웃과 다퉜지만 경찰에 입건된 건 이웃집에 간장을 뿌려 재물손괴 혐의를 받았을 때뿐입니다 경찰은 당시 현장 출동 경찰관이 안 씨가 조현병 환자라는 사실을 몰랐다고 밝혔습니다 아파트 주민과 관리사무소 측은 안 씨가 이웃집과 승강기에 인분을 뿌리는 갈등이 잦았다고 말했습니다 불안을 느낀 주민이 설치한 CCTV에 안 씨가 18살 여학생을 쫓아 집에 침입하려던 모습도 포착됐는데요 해당 여학생은 사건으로 숨졌습니다 경찰은 피해자들에 대해 1 1 보호팀을 배치하겠다고 밝혔습니다
Output: 경남 진주의 아파트 방화 사건 피의자는 지상 40대 남성인데요 자신의 집에 불을 낸 뒤 뒤집혀 이웃들을 흉기로 공격했는데 최근까지 박격포 갈등이 부러져든 것으로 알려졌습니다 경찰은 안 씨와 관련해 최근 7건의 경찰 구

Input: 최근 포스코에서 민주노총 산하의 새 노동조합이 출범한 가운데 포스코 사측이 노조 와해 공작을 벌였다는 주장이 제기됐습니다 정의당 추혜선 의원은 오늘 오후 2시 국회 회견을 열고 그제 포스코 노무협력실 직원들이 작성 중이던 문건과 참석자의 메모를 공개했습니다 문건에는 강성노조가 근로자 권익과 무관한 활동을 추진하고 민주노총에 대한 부정적 평가와 직원들의 카카오톡 오픈채팅방 대화를 인용하며 조직 와해 우려 발언 정치색을 띈 의견 지속적 등록 이라고 분석한 내용도 일부 포함돼 있습니다 포스코를 사랑하는 직원 명의로 작성된 직원 대상 호소문에도 굴지의 기업들이 치킨게임과 같은 노사대결 구도 속에서 어려움을 겪었던 사례를 눈으로 직접 왔다 면서 결과에 대한 피해는 고스란히 직원들이 안아야 한다 노조 활동에 대한 부정적 평가가 적혀 있습니다 참석자의 메모 가운데서는 우리가 만든 논리가 일반 직원에게 전달되는지 안 되는지 확인 시범부서를 선정해 조직화 행정부소장 또는 제철소장이 해야 미션 분명히 줘야 라는 내용이 발견됐습니다 추 의원은 포스코에서 최고위층의 지시 내지 동의 하에 종합적인 노조 무력화 대책이 마련되고 있다는 것이 문건들로 확인됐다 면서 노조와 대화하겠다고 했던 최정우 회장은 책임 있는 답변을 내놔야 한다 고 주장했습니다 이에 대해 포스코 측은 노조 와해 공작 주장은 사실 무근이며 회사가 노조 동향을 모니터링하는 것은 합법적이고 통상적인 업무 라면서 호소문도 회사가 작성한 게 아니다 라고 해명했습니다 사무실에 들어와 회의 문건을 가져간 노조원들에 대해선 경찰 조사가 진행 중이며 회사도 사규에 조치할 예정이라고 덧붙였습니다
Output: 정의당 추혜선 의원은 오늘 오후 2시 국회 회견을 열고 그제 포스코지회선거실 직원들이 작성 중이던 문건과 참석자의 다수가 공개했습니다 이에 대해 포스코 측은 노조 와해 공작 이후에도 사실 세부 소속 직원에게 전달되려면 안 되는지 확인 시범부서를 개정하는 또는 보도는 또는 보도는 미션 있는 답변을 진행 중이며 회사도 

Input: 마약 투약 혐의를 받고 있는 남양유업 창업주의 외손녀 황하나 씨가 경찰에 붙잡혔습니다 경찰은 황 씨가 입원 중인 병원을 찾아가 체포영장을 집행했습니다 보도에 입니다 마약 투약 의혹으로 경찰 수사를 받고 있는 남양유업 창업주의 외손녀 황하나씨가 오늘 오후 경찰에 체포됐습니다 황 씨는 체포 당시 성남시 분당 서울대병원에서 입원해 있었으며 체포영장을 발부 받은 수사관들이 황씨를 붙잡아 압송했습니다 경찰은 지난해 10월 황 씨의 마약 투약 의혹과 관련한 첩보를 입수해 수사 중이었습니다 해당 첩보에는 황 씨의 과거 필로폰 투약 혐의는 물론이고 마약 관련 혐의도 포함돼 있는 것으로 전해졌습니다 경찰은 이후 두 차례에 걸쳐 황 씨에 대한 압수수색 영장을 신청했습니다 사건을 지휘하는 수원지방검찰청은 황 씨가 마약을 투약한 지 수년이 지나 압수수색 영장 집행이 어렵다며 이를 반려해 논란이 일었습니다 이후에도 황 씨는 경찰의 출석 요구에 응하지 않았습니다 황 씨는 지난 2015년 대학생 조 모씨에게 필로폰 0 5그램을 공급하고 세 차례 투약해 준 혐의로 입건됐지만 당시 서울 종로경찰서가 황 씨를 한 번도 소환 조사하지 않고 무혐의로 검찰에 송치한 것으로 확인됐습니다 오늘 황 씨가 체포됨에 수사도 급물살을 탈 것으로 보입니다 서울지방경찰청 지능범죄수사대는 과거 황씨의 수사 과정에 문제가 없었는지 내사에 착수해 조사를 벌이고 있습니다
Output: 마약 투약 혐의를 받고 있는 남양유업 창업주의 외손녀 황하나 씨가 경찰에 붙잡혔습니다 경찰은 지난해 10월 황 씨의 마약 투약 의혹과 관련한 첩보를 입수해 수사 중이었습니다 경찰은 이후 두 차례에 걸쳐 황 씨에 대한 압수수색 영장을 신청했습니다
BLEU: 1.726932402073141
Rouge F1: 38.32335298361362
Rouge Recall: 23.703703703703706
Rouge Precision: 100.0

Input: 어제 울산 울주군에서 발생한 산불이 이틀째 계속되면서 주민 4천여명이 대피했습

Input: 오늘 부산의 한 폐수처리 공장에서 유독성 물질이 누출돼 작업하던 직원 4명이 의식을 잃었습니다 작업자들은 아무런 보호장비를 착용하지 않은 채 사고를 당한 것으로 드러났습니다 입니다 폐수처리 공장 안에서 소방 특수대원들이 황화수소에 오염된 시설 내부를 물로 씻어내고 있습니다 오늘 오후 1시쯤 이곳에서 황화수소가 누출되면서 업체 직원 7명이 가스에 질식돼 병원으로 옮겨졌습니다 3명은 경상이지만 4명은 호흡과 맥박을 회복했을 뿐 아직 의식을 회복하지는 못했습니다 경찰과 소방당국은 폐수를 집수조에 옮겨 담는 과정에서 사고가 난 것으로 추정하고 있습니다 소방당국이 사고 발생 한 뒤에 측정한 황화수소 농도는 150ppm에 달했습니다 100에서 300ppm일 경우 구토와 어지러움에 이어 후각 마비와 질식 위험이 있고 700ppm 경우 호흡정지에 이어 사망에 이를 수도 있습니다 경찰 조사 결과 사고 당시 작업자들은 방독면과 장갑 기본적인 안전 장비조차 착용하지 않았던 것으로 확인됐습니다 경찰은 공장 관계자를 불러 산업안전보건법에 근거해 안전수칙을 지켰는지와 환기시설을 제대로 설치했는지 등을 조사할 방침입니다
Output: 오늘 부산의 한 폐수처리 공장에서 유독성 물질이 누출돼 작업하던 직원 4명이 의식을 잃었습니다 오늘 오후 1시쯤 이곳에서 네팔인 시설에 누출되면서 업체 직원 7명이 가스에 질식돼 병원으로 옮겨졌습니다
BLEU: 1.6095335841902998
Rouge F1: 32.85714256377552
Rouge Recall: 20.0
Rouge Precision: 92.0

Input: 화재 원인 조사를 위한 합동 감식이 오늘 오전부터 현재까지 진행되고 있습니다 입니다 대한송유관공사 저유소 탱크 화재의 원인을 밝히기 위한 합동 감식이 오늘 오전부터 시작됐습니다 국립과학수사연구원과 소방당국 가스 전기안전공사 등이 참여했습니다 오늘 합동 현장 감식은 설비 결함이나 오작동이 있었는지와 외부적인 요인이 있는지 폭발 원인을 규명하는 데 집중될 예정입니다 화재 

...

Input: 편의점 직원으로 위장 취업해 점주가 자리를 비운 사이 현금과 상품권 등을 훔쳐 달아난 30대 남성이 경찰에 붙잡혔습니다 서울 도봉경찰서는 서울과 경기도 일대 편의점에서 천 2백여만 원을 훔쳐 달아난 36살 남성 한 씨를 상습절도 등의 혐의로 붙잡았다고 오늘 밝혔습니다 한 씨는 지난 3월부터 서울 강북구와 도봉구 경기도 의정부시와 화성시 등의 9개 편의점에서 현금과 상품권 총 천 2백여만 원어치 금품을 훔쳐 달아난 혐의를 받고 있습니다 한 씨는 아르바이트생들이 꺼리는 야간 근무를 할 수 있다고 하는 등의 방법으로 점주의 호감을 편의점에 일을 시작한 뒤 2 5시간 만에 범행을 저질렀습니다 일을 시작한 지 채 5분도 되지 않아 범행을 저지른 적도 있는 것으로 알려졌습니다 경찰 조사 결과 한 씨는 훔친 돈을 생활비와 유흥비 등에 쓴 것으로 드러났습니다 한 씨는 자신의 주거지에 돌아가지 않고 여관이나 찜질방을 전전하거나 일회용 교통권을 사용하는 방법으로 경찰의 추적을 피한 것으로 알려졌습니다
Output: 한 씨는 지난 3월부터 서울 수서경찰서는구와 지하주차장에서 경기도 의정부시와 화성시 등의 9개 편의점에서 현금과 상품권 총 천 천 2백여만 원어치 금품을 훔쳐 달아난 혐의를 받고 있습니다
BLEU: 1.9737106746531266
Rouge F1: 35.93749968566895
Rouge Recall: 22.330097087378643
Rouge Precision: 92.0

Input: 광화문 미국대사관 외벽에 검은 현수막이 걸렸습니다 흑인 목숨도 소중하다 미 전역을 휩쓴 인종차별 반대 시위의 핵심 구호입니다 주한미국대사관은 SNS를 통해 평화로운 시위를 지지한다 며 현수막이 인종 차별과 경찰 만행에 대한 항의 뜻한다고 밝혔습니다 해리스 대사도 SNS로 힘을 보탰습니다 흑인 민권 운동가 마틴 루터 킹 목사의 추도사를 인용하며 다양성으로부터 우리는 힘을 얻는다 고 했습니다 이처럼 대사관 차원에서 시위 지지에 나선 건 전 세계 미국 재외공관 가운데 주한

Input: 최근 음주운전에 대한 처벌을 강화해야 한다는 여론이 높은데요 어제 광주에서 음주운전으로 면허가 취소된 30대 남성이 행인을 치고 도주한 사건이 있었습니다 시민의 신고로 운전자는 붙잡혔지만 피해자는 목숨을 잃었습니다 보도합니다 승용차 조수석 앞 유리가 깨져 있습니다 음푹 패인 채 곳곳에 금이 갔습니다 33살 박 씨는 자신의 차를 운전하다 어제 오후 7시 반쯤 광주광역시의 한 도로에서 70살 김 씨를 치었습니다 박 씨는 곳에서 길을 건너던 김 씨를 들이받고도 조치 없이 그대로 도주했습니다 김 씨는 택시 기사의 신고로 병원에 이송됐지만 끝내 숨졌습니다 주변 CCTV나 사고 당시 목격자가 없어 수사가 어려운 상황 앞 유리가 깨진 차량이 도로를 달리는 걸 본 한 시민이 친구인 경찰에게 알려 덜미가 잡혔습니다 경찰 조사 결과 박 씨는 음주운전 삼진아웃 지난해 11월 면허가 취소된 상태였습니다 박 씨는 면허 재취득 기간을 줄이기 위해 음주운전 예방교육을 받으러 전남 완도에서 광주로 오던 중 사고를 낸 뒤 겁이 도주했다고 진술했습니다 사고 당시 음주 상태는 아니었던 것으로 확인됐습니다 경찰은 박 씨에 대해 도주 치사 혐의로 구속영장을 신청했습니다
Output: 어제 광주에서 음주운전으로 면허가 취소된 30대 남성이 행인을 치고 도주한 사건이 있었습니다 사고 당시 음주 상태는 아니었던 것으로 확인됐습니다 사고 당시 음주 상태는 아니었던 것으로 확인됐습니다
BLEU: 1.0922196998267393
Rouge F1: 25.50335548218549
Rouge Recall: 14.615384615384617
Rouge Precision: 100.0

Input: 요양병원 등을 불법으로 설립해 운영하면서 천억 원이 넘는 건강보험급여를 타낸 의료재단과 생협이 경찰에 적발됐습니다 직원으로 등록한 가족에게 고액의 급여를 주고 외제차를 구입해 자녀에게 주기도 했지만 관리 감독은 이뤄지지 않았습니다 보도합니다 부산의 한 의료재단이 운영하는 요양병원입니다 재단은 12년 요양병원

Input: 금속노조는 현대자동차 비정규직 판매노동자가 노조에 가입했다는 이유로 폭행과 해고를 당했다며 빨리 수사를 하라고 요구했습니다 전국금속노동조합은 오늘 오후 청와대 회견을 열고 또다시 어이없는 일이 발생했다 며 현대자동차 남안산대리점의 비정규직 판매사원에 대한 폭행과 해고를 규탄했습니다 금속노조는 지난 21일 현대자동차 남안산대리점의 비정규직 판매사원이 노조에 가입했다는 대리점 소장에게 폭행을 당하고 절차 없이 해고를 당했다고 주장했습니다 금속노조는 과정에서 경찰은 가해자인 대리점 소장은 가만히 두고 피해자에게 수갑을 채워 불법적으로 강제 연행했다 며 말로만 인권경찰이라 하지 말고 동등한 인격체로 노동자를 대하라 고 말했습니다 이어 이들은 그동안 현대기아차는 노동조합에 가입한 대리점을 7개나 폐업시키고 조합원들에게 끊임없는 탈퇴종용과 부당해고를 해왔다 며 증거들을 모아 6개월 전에 서울중앙지검에 현대 기아차 원청을 고소했지만 서울고용노동청은 6개월이 넘는 단 한 명의 참고인 조사조차 하지 않은 채 수사를 지연하고 고 항의했습니다 금속노조는 문재인 정부의 노동 존중이 것인가
Output: 금속노조는 지난 21일 모가 남안산마트 107 비정규직 판매사원이 탑승을 가입했다는 대리점 소장에게 폭행을 당하고 절차 없이 증거불충 당했다고 주장했습니다
BLEU: 0.36515499350585684
Rouge F1: 23.140495603032583
Rouge Recall: 13.725490196078432
Rouge Precision: 73.68421052631578

Input: 이른바 알몸남 사건으로 홍역을 치른 동덕여대가 어제부터 외부인 출입을 제한했습니다 학생들은 안전을 위해 어쩔 수 없다는 입장을 보였지만 이렇게까지 해야하냐는 씁쓸한 반응도 나왔습니다 보도합니다 동덕여대 강의실에서 찍은 한 남성의 음란 사진 SNS를 통해 유포되면서 학교가 발칵 뒤집혔습니다 이른바 알몸남 은 경찰 신고 이틀 만에 체포됐지만 안심하고 학교를 다닐 수 없다는 학생들의 반발이 이

Input: 코로나19 지역 감염이 없던 태국에 비상이 걸렸습니다 한 마을에서 확진자 한 명이 발생하더니 다음날 같은 마을에서 548명이 감염된 것으로 확인됐습니다 정부는 마을을 통째로 봉쇄했습니다 직접 다녀왔습니다 봉쇄 나흘쨉니다 방콕 인근에서 가장 큰 어시장을 철조망이 둘러쌌습니다 지난 금요일 한 새우 도매상이 확진 판정을 받았고 다음날은 마을에서 548명이 확진 판정을 받았습니다 대부분 미얀마에서 온 이주 노동자들입니다 다음날 마을은 전격 봉쇄됐습니다 고립된 이들을 위해 식품과 물이 계속 들어갑니다 미얀마 이민자 남매 집에서 나온 사이에 봉쇄된 마을에 갇힌 아버지와 떨어졌습니다 마을에 남겨진 사람들은 사실상 바이러스 감염에 무방비 상탭니다 봉쇄된 마을 주변으로 10여 곳에 임시 검사소가 세워졌습니다 확진자가 수백 명 나왔단 소식에 놀란 주민들의 줄이 끝없이 이어집니다 마을은 봉쇄됐지만 이미 감염이 확산되면서 오늘까지 사흘동안 확진자가 무려 1 303명이 발생했습니다 관건은 방콕으로의 확산을 막는 겁니다 곳곳에 이민자를 차단하는 검문소가 설치됐고 사뭇사콘주 전체에 밤 10시부터 야간 통행금지도 시작됐습니다 사뭇 사콘의 교민은 30명 정돕니다 사뭇 사콘에서 뉴스 김원장입니다 촬영
Output: 지난 금요일 한 산간 도매상이 확진 판정을 받았고 다음날 같은 마을에서 548명이 감염된 것으로 확인됐습니다 주민들은 사망자가 급증하면서 고립된 이들을 나온 사이에 발견되면서 마을에 갇힌 사건입니다 주민들은 감염이 확산되면서 오늘까지 사흘동안 확진자가 무려 1 303명이 발생했습니다
BLEU: 4.888645395514294
Rouge F1: 29.629629290885536
Rouge Recall: 18.89763779527559
Rouge Precision: 68.57142857142857

Input: 필리핀 세부에서 20대 한국인 남성이 총격을 받아 숨졌습니다 외교부 당국자는 현지시간으로 어제 오후 6시 17분쯤 필리핀 세부시 소재 프린스코트 모텔 2층 복도에서

Input: 문재인 대통령이 2차 북미 정상회담이 직후인 다음달 초에 최대 7개 부처 장관을 바꾸는 중폭 개각을 단행할 것으로 보입니다 청와대가 최근 민주당 박영선 우상호 의원에 대한 검증에 착수한 것으로 알려지면서 현역 의원들의 입각 가능성에도 관심이 쏠립니다 보도입니다 개각 대상으로 거론되는 부처는 주로 내년 총선에 출마할 정치인 출신 장관들이 있는 곳입니다 김부겸 행정안전부 김영춘 해양수산부 김현미 국토교통부 도종환 문화체육관광부 장관 국회의원은 아니지만 출마가 유력한 유영민 과기정통부 장관은 교체가 유력합니다 조명균 통일부 장관은 일찌감치 사의를 표명했고 정부업무평가에서 미흡 평가를 받은 박상기 법무장관도 교체 가능성이 큰 것으로 알려졌습니다 가운데 청와대가 최근 민주당 박영선 우상호 의원에 대한 검증을 경찰에 의뢰한 것으로 알려져 여당 의원들이 총선 출마를 접고 입각하게 될지도 관심입니다 박 의원은 법무부 장관 우 의원은 문체부 장관을 염두에 두고 검증 대상에 올린 것으로 보입니다 통일부장관엔 김연철 통일연구원장이 유력하게 거론되고 행안부 장관은 홍미영 더불어민주당 다문화위원장 등이 물망에 올랐습니다 청와대 관계자는 개각에는 총선 출마자 배제 원칙이 적용돼 주로 관료 교수 출신들이 검토되고 정치인 입각 규모는 크지 않을 이라고 말했습니다 국민들이 체감할 수 있는 성과 창출 초첨을 맞춘 개각의 시기는 2차 북미 정상회담이 끝난 뒤인 다음달 초로 예상됩니다 청와대 관계자는 후보자 검증이 마무리 단계지만 검증 과정에서 한 두명의 결격 사유가 발견돼 재추천 절차를 진행하고 있다고 전했습니다
Output: 문재인 대통령이 2차 북미 정상회담이 직후인 다음달 초에 최대 7개 부처 장관을 바꾸는 중폭공원을 바꾸는 중폭 개각을 단행할 것으로 보입니다 청와대 관계자는 인사청문회에는 총선 출마자 배제 원칙이 투입되는 주로 관료 교수 출신들이 남한 국가균형발전위원장을 명단에 않을 이라고 말했습니다 청와대 관계자는 후보자 논문에 검증사 배제시키면서 지금의 우리나라 관료 교

target data (추출 요약문과 비교)

#### ROUGE-L Score  
reference : gensim summarize 라이브러리를 이용해 추출한 추출 요약문    
produced summary : RC-Transformer 생성 요약문 

In [ ]:
rouge = Rouge()

generated_summary_target_dist = pd.DataFrame(columns=['Origin Contents', 'Target Summary', 'Generated Summary', 
                                                'ROUGE F1', 'ROUGE Recall', 'ROUGE Precision'])
for inp, outp in zip(input_test, output_test):
    decoder_src = Decoder(option)
    input_sent = decoder_src.decode(inp)
    target_sent = decoder_src.decode(outp)
    
    predict_summary = predict(inp)
    
    print('Input : {}'.format(input_sent))
    print('Target : {}'.format(target_sent))
    print('Output : {}'.format(predict_summary))
    
    rouge_scores = rouge.get_scores(target_sent, input_sent)
    rouge_f = rouge_scores[0]['rouge-l']['f']
    print('Rouge F1: {}'.format(rouge_f * 100))
    
    rouge_r = rouge_scores[0]['rouge-l']['r']
    print('Rouge Recall: {}'.format(rouge_r * 100))
    
    rouge_p = rouge_scores[0]['rouge-l']['p']
    print('Rouge Precision: {}\n'.format(rouge_p * 100))
    
    summary = {'Origin Contents' : input_sent, 'Target Summary' : target_sent,'Generated Summary' : predict_summary,
               'ROUGE F1': rouge_f, 'ROUGE Recall': rouge_r, 'ROUGE Precision': rouge_p}
    
    generated_summary_target_dist = generated_summary_target_dist.append(summary, ignore_index=True)
    

In [ ]:
import nltk.translate.bleu_score as bleu
from rouge import Rouge 
rouge = Rouge()

validation_generated_dist = pd.DataFrame(columns=['Origin Contents', 'Generated Summary', 
                                            'ROUGE F1', 'ROUGE Recall', 'ROUGE Precision'])

for _, val_proc_path in enumerate(iglob(os.path.join(VAL_PREPROCESSED_PATH, '**.csv'), recursive=False)):

    media_name = get_media_name(val_proc_path)
    val_summary_path = os.path.join(VAL_SUMMARY_PREPROCESSED_PATH, media_name +".csv")
    print(media_name, val_proc_path)
    
    f_src = open(val_proc_path, 'r', newline="\n", encoding="utf-8")
    f_tar = open(val_summary_path, 'r', newline="\n", encoding="utf-8")
        
    for [_, title, contents], [_, _, target] in zip(csv.reader(f_src), csv.reader(f_tar)):
        content = contents.split("\t")
        target_summary = target.split("\t")
        
        encoder = IntergerEncoder(options=option, filepaths=None)
        
        input_sent = ' '.join(content)
        input_enc_sent = START_TOKEN + encoder.line_encoder(content_line) + END_TOKEN
        
        predict_summary = predict(input_enc_sent)
        target_summary = ' '.join(target_summary)
    
        print('Input: {}'.format(input_sent))
        print('Output: {}'.format(predict_summary))
    
        rouge_scores = rouge.get_scores(predict_summary, input_sent)
        rouge_f = rouge_scores[0]['rouge-l']['f']
        print('Rouge F1: {}'.format(rouge_f * 100))
    
        rouge_r = rouge_scores[0]['rouge-l']['r']
        print('Rouge Recall: {}'.format(rouge_r * 100))
    
        rouge_p = rouge_scores[0]['rouge-l']['p']
        print('Rouge Precision: {}\n'.format(rouge_p * 100))
    
        summary = {'Origin Contents' : input_sent, 'Generated Summary' : predict_summary,
               'ROUGE F1': rouge_f, 'ROUGE Recall': rouge_r, 'ROUGE Precision': rouge_p}
    
        validation_generated_dist = validation_generated_dist.append(summary, ignore_index=True)
    

In [ ]:
ref = '경남 진주의 아파트 방화 사건 피의자는 주민인 40대 남성인데요 자신의 집에 불을 낸 뒤 대피하던 이웃들을 흉기로 공격했는데 최근까지 이웃과 갈등이 잦았던 것으로 확인됐습니다 현장 연결해 수사상황 알아보겠습니다 현재 수사가 진행되고 있습니까 저는 지금 수사본부가 마련된 경남 진주경찰서에 나와 있습니다 현재 경찰서에서는 아파트 방화 살인사건 피의자 42살 안 모씨에 대한 조사가 진행 중인데요 안 씨는 범행을 일부 시인했지만 자세한 사건 경위와 범행 동기를 두고 횡설수설하며 제대로 진술하지 않는 것으로 알려졌습니다 경남지방경찰청은 진주경찰서장을 총괄로 한 특별 수사단을 꾸려 형사 전원을 투입해 수사 중인데요 경찰 조사 결과 피의자 안 씨는 살았고 지난 2015년 조현병 증세로 치료받은 것으로 확인됐습니다 경찰은 프로파일러를 투입해 안 씨의 심리 상태를 파악하는 한편 안 씨에 대해 방화와 살인 살인미수 혐의로 구속영장을 신청할 예정입니다 피의자가 이웃과 마찰이 잦아 경찰도 여러 번 출동했다고 하던데 제대로 조치되지 않은 건가요 경찰은 안 씨와 관련해 최근 7건의 경찰 출동이 있었고 가운데 5건이 해당 아파트에서 벌어졌다고 밝혔는데요 층간소음이나 벌레가 떨어진다며 이웃과 다퉜지만 경찰에 입건된 건 이웃집에 간장을 뿌려 재물손괴 혐의를 받았을 때뿐입니다 경찰은 당시 현장 출동 경찰관이 안 씨가 조현병 환자라는 사실을 몰랐다고 밝혔습니다 아파트 주민과 관리사무소 측은 안 씨가 이웃집과 승강기에 인분을 뿌리는 갈등이 잦았다고 말했습니다 불안을 느낀 주민이 설치한 CCTV에 안 씨가 18살 여학생을 쫓아 집에 침입하려던 모습도 포착됐는데요 해당 여학생은 사건으로 숨졌습니다 경찰은 피해자들에 대해 1 1 보호팀을 배치하겠다고 밝혔습니다'
pred= '경남 남편을 살해한 혐의로 구속된 30대 남성이 살해한 혐의로 붙잡혔습니다 경찰은 범행 혐의로 조사를 받고 있습니다 경찰은 CCTV에 대해 범행 혐의로 조사를 받았습니다 경찰은 경찰은 CCTV 등을 토대로 범행 혐의로 조사를 받았습니다'

ref_pcs = [sp_src.encode_as_pieces(ref)]
pred_pcs = [sp_src.encode_as_pieces(pred)]

print(f'reference: {ref_pcs}')
print('candidate:', pred_pcs)
bleu.sentence_bleu([ref], pred)


In [ ]:
scores = rouge.get_scores(pred, ref)
scores[0]['rouge-l']['f']

In [ ]:
saveCSVFile(PREDICT_PATH, 'KBS', generated_summary_dist)

In [ ]:
saveCSVFile(PREDICT_PATH, 'KBS', generated_summary_target_dist)